In [1]:
!pip install -q --upgrade pip
!pip install -q seaborn

In [2]:
from __future__ import print_function
from sklearn.preprocessing import LabelEncoder
from keras import callbacks
from sklearn.preprocessing import Normalizer
import pandas as pd
import numpy as np
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
np.random.seed(1337)  # for reproducibility


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
# Seed all random number generators
def seed_everything(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
def get_strategy(device='GPU'):
    IS_TPU = False
    if "TPU" in device:
        tpu = 'local' if device=='TPU-VM' else None
        print("connecting to TPU...")
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect(tpu=tpu)
        strategy = tf.distribute.TPUStrategy(tpu)
        IS_TPU = True

    if device == "GPU"  or device=="CPU":
        ngpu = len(tf.config.experimental.list_physical_devices('GPU'))
        if ngpu>1:
            print("Using multi GPU")
            strategy = tf.distribute.MirroredStrategy()
        elif ngpu==1:
            print("Using single GPU")
            strategy = tf.distribute.get_strategy()
        else:
            print("Using CPU")
            strategy = tf.distribute.get_strategy()

    if device == "GPU":
        print("Num GPUs Available: ", ngpu)

    AUTO     = tf.data.experimental.AUTOTUNE
    REPLICAS = strategy.num_replicas_in_sync
    print(f'REPLICAS: {REPLICAS}')
    
    return strategy, REPLICAS, IS_TPU

STRATEGY, N_REPLICAS, IS_TPU = get_strategy()

Using single GPU
Num GPUs Available:  1
REPLICAS: 1


In [4]:
col_names = [
    'sport', 'dsport', 'proto', 'state', 'dur', 'sbytes', 'dbytes', 'sttl', 'dttl', 'sloss',
    'dloss', 'service', 'Sload', 'Dload', 'Spkts', 'Dpkts', 'swin', 'dwin', 'stcpb', 'dtcpb', 'smeansz', 'dmeansz',
    'trans_depth', 'res_bdy_len', 'Sjit', 'Djit', 'Sintpkt', 'Dintpkt', 'tcprtt', 'synack',
    'ackdat', 'is_sm_ips_ports', 'ct_state_ttl', 'ct_flw_http_mthd', 'is_ftp_login', 'ct_ftp_cmd', 'ct_srv_src',
    'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm',
    'Label'
]

In [5]:
TRAIN_FILE = glob.glob('/kaggle/input/unsw-nb15/csv_fold/fold*.csv')

In [6]:
TRAIN_FILE

['/kaggle/input/unsw-nb15/csv_fold/fold_4.csv',
 '/kaggle/input/unsw-nb15/csv_fold/fold_19.csv',
 '/kaggle/input/unsw-nb15/csv_fold/fold_11.csv',
 '/kaggle/input/unsw-nb15/csv_fold/fold_15.csv',
 '/kaggle/input/unsw-nb15/csv_fold/fold_17.csv',
 '/kaggle/input/unsw-nb15/csv_fold/fold_21.csv',
 '/kaggle/input/unsw-nb15/csv_fold/fold_12.csv',
 '/kaggle/input/unsw-nb15/csv_fold/fold_0.csv',
 '/kaggle/input/unsw-nb15/csv_fold/fold_18.csv',
 '/kaggle/input/unsw-nb15/csv_fold/fold_2.csv',
 '/kaggle/input/unsw-nb15/csv_fold/fold_5.csv',
 '/kaggle/input/unsw-nb15/csv_fold/fold_20.csv',
 '/kaggle/input/unsw-nb15/csv_fold/fold_10.csv',
 '/kaggle/input/unsw-nb15/csv_fold/fold_6.csv',
 '/kaggle/input/unsw-nb15/csv_fold/fold_14.csv',
 '/kaggle/input/unsw-nb15/csv_fold/fold_1.csv',
 '/kaggle/input/unsw-nb15/csv_fold/fold_16.csv',
 '/kaggle/input/unsw-nb15/csv_fold/fold_13.csv',
 '/kaggle/input/unsw-nb15/csv_fold/fold_7.csv',
 '/kaggle/input/unsw-nb15/csv_fold/fold_8.csv',
 '/kaggle/input/unsw-nb15/cs

In [7]:
TEST_FILE = glob.glob('/kaggle/input/unsw-nb15/csv_fold/test.csv')

In [8]:
TEST_FILE

['/kaggle/input/unsw-nb15/csv_fold/test.csv']

In [9]:
def load_csv(name):
    df = pd.read_csv(name, header=None, names=col_names, low_memory=False)
    df = df.iloc[1:]
    df = df[~((df['is_sm_ips_ports'] == 0)|(df['is_sm_ips_ports'] == 1))]
    df = df[~((df['is_ftp_login'] == 0)|(df['is_ftp_login'] == 1))]
    df = df[~((df['Label'] == 0)|(df['Label'] == 1))]
    return df

In [10]:
traindata = []
for name in (TRAIN_FILE):
    traindata.append(load_csv(name))
traindf = pd.concat(traindata)

In [11]:
traindf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2100000 entries, 1 to 100000
Data columns (total 44 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   sport             object
 1   dsport            object
 2   proto             object
 3   state             object
 4   dur               object
 5   sbytes            object
 6   dbytes            object
 7   sttl              object
 8   dttl              object
 9   sloss             object
 10  dloss             object
 11  service           object
 12  Sload             object
 13  Dload             object
 14  Spkts             object
 15  Dpkts             object
 16  swin              object
 17  dwin              object
 18  stcpb             object
 19  dtcpb             object
 20  smeansz           object
 21  dmeansz           object
 22  trans_depth       object
 23  res_bdy_len       object
 24  Sjit              object
 25  Djit              object
 26  Sintpkt           object
 27  Dintpkt      

In [12]:
traindf.head

<bound method NDFrame.head of         sport dsport proto state       dur sbytes dbytes sttl dttl sloss  ...  \
1        4313  33594   tcp   FIN  0.041962   2438  19138   31   29     7  ...   
2       54015   5190   tcp   FIN  0.004304   1036   2262   31   29     4  ...   
3       55256     25   tcp   FIN  0.426766  37996   3276   31   29    19  ...   
4       57213     21   tcp   FIN  0.022451   2934   3742   31   29    11  ...   
5        1679  27326   tcp   FIN  0.029956   3406  40478   31   29     7  ...   
...       ...    ...   ...   ...       ...    ...    ...  ...  ...   ...  ...   
99996    5551     25   tcp   FIN  0.036674  37262   3380   31   29    18  ...   
99997   60838    111   udp   CON  0.004771    568    320   31   29     0  ...   
99998    1665    143   tcp   FIN  0.044128   7816  14952   31   29    30  ...   
99999   63494     80   tcp   FIN  1.076023   1684  10168   31   29     3  ...   
100000  53128     25   tcp   FIN  0.070535  37262   3380   31   29    18  ...  

In [13]:
traindf.isna().sum()

sport                    0
dsport                   0
proto                    0
state                    0
dur                      0
sbytes                   0
dbytes                   0
sttl                     0
dttl                     0
sloss                    0
dloss                    0
service                  0
Sload                    0
Dload                    0
Spkts                    0
Dpkts                    0
swin                     0
dwin                     0
stcpb                    0
dtcpb                    0
smeansz                  0
dmeansz                  0
trans_depth              0
res_bdy_len              0
Sjit                     0
Djit                     0
Sintpkt                  0
Dintpkt                  0
tcprtt                   0
synack                   0
ackdat                   0
is_sm_ips_ports          0
ct_state_ttl             0
ct_flw_http_mthd    941659
is_ftp_login        996465
ct_ftp_cmd               0
ct_srv_src               0
c

In [19]:
traindf.fillna(-1, inplace=True)

In [15]:
traindf.isna().sum()

sport               0
dsport              0
proto               0
state               0
dur                 0
sbytes              0
dbytes              0
sttl                0
dttl                0
sloss               0
dloss               0
service             0
Sload               0
Dload               0
Spkts               0
Dpkts               0
swin                0
dwin                0
stcpb               0
dtcpb               0
smeansz             0
dmeansz             0
trans_depth         0
res_bdy_len         0
Sjit                0
Djit                0
Sintpkt             0
Dintpkt             0
tcprtt              0
synack              0
ackdat              0
is_sm_ips_ports     0
ct_state_ttl        0
ct_flw_http_mthd    0
is_ftp_login        0
ct_ftp_cmd          0
ct_srv_src          0
ct_srv_dst          0
ct_dst_ltm          0
ct_src_ltm          0
ct_src_dport_ltm    0
ct_dst_sport_ltm    0
ct_dst_src_ltm      0
Label               0
dtype: int64

In [16]:
traindf.shape

(2100000, 44)

In [17]:
traindf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2100000 entries, 1 to 100000
Data columns (total 44 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   sport             object
 1   dsport            object
 2   proto             object
 3   state             object
 4   dur               object
 5   sbytes            object
 6   dbytes            object
 7   sttl              object
 8   dttl              object
 9   sloss             object
 10  dloss             object
 11  service           object
 12  Sload             object
 13  Dload             object
 14  Spkts             object
 15  Dpkts             object
 16  swin              object
 17  dwin              object
 18  stcpb             object
 19  dtcpb             object
 20  smeansz           object
 21  dmeansz           object
 22  trans_depth       object
 23  res_bdy_len       object
 24  Sjit              object
 25  Djit              object
 26  Sintpkt           object
 27  Dintpkt      

In [18]:
def find_columns_with_mixed_datatypes(df):
    mixed_datatype_columns = []

    for column in df.columns:
        unique_dtypes = df[column].apply(type).unique()
        if len(unique_dtypes) > 1:
            mixed_datatype_columns.append(column)

    return mixed_datatype_columns
mixed_datatype_cols = find_columns_with_mixed_datatypes(traindf)
print("Columns with mixed datatypes:", mixed_datatype_cols)


Columns with mixed datatypes: ['ct_flw_http_mthd', 'is_ftp_login']


In [ ]:
plt.figure(figsize=(8,8))
ax = sns.barplot(x='ackdat',y='counts',data=traindf.groupby('ackdat').size().reset_index(name='counts'))
ax.set_title('Distribution of attack labels')
ax.set_xlabel('is_ftp_login')
ax.set_ylabel('Count')
plt.xticks(rotation=45)
plt.yticks(rotation=45)
plt.show()

In [ ]:
testdf = load_csv(TEST_FILE[0])

In [ ]:
testdf.info()

In [ ]:
testdf.fillna(-1, inplace=True)
testdf.isna().sum()

In [ ]:
testdf.shape

In [ ]:
testdf.info()

In [ ]:
plt.figure(figsize=(8,8))
ax = sns.barplot(x='is_ftp_login',y='counts',data=traindf.groupby('is_ftp_login').size().reset_index(name='counts'))
ax.set_title('Distribution of attack labels')
ax.set_xlabel('is_ftp_login')
ax.set_ylabel('Count')
plt.xticks(rotation=45)
plt.yticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(8,8))
ax = sns.barplot(x='Label',y='counts',data=traindf.groupby('Label').size().reset_index(name='counts'))
ax.set_title('Distribution of attack labels')
ax.set_xlabel('Label')
ax.set_ylabel('Count')
plt.xticks(rotation=45)
plt.yticks(rotation=45)
plt.show()

In [ ]:
checkpointer = callbacks.ModelCheckpoint(
    filepath="./content/results/cnn3results/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='accuracy', mode='max')
csv_logger = CSVLogger(
    './content/results/cnn3results/cnntrainanalysis1.csv', separator=',', append=True)

In [ ]:
numeric_columns_train = traindf.select_dtypes(include=[np.number])
numeric_columns_test = testdf.select_dtypes(include=[np.number])


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Normalize the numeric columns in testdf
normalized_testdf = testdf.copy()
normalized_testdf[numeric_columns_test.columns] = scaler.transform(numeric_columns_test)


In [ ]:
# Convert 0.0 to 0 in all columns
filled_traindf = traindf.copy()
# Convert numeric columns to int
numeric_columns = filled_traindf.select_dtypes(include=[np.number]).columns
filled_traindf[numeric_columns] = filled_traindf[numeric_columns].astype(int)


# Normalize the numeric columns using z-score normalization
from sklearn.preprocessing import StandardScaler

# Select numeric columns
numeric_columns = filled_traindf.select_dtypes(include=[np.number])

# Create a StandardScaler object
scaler = StandardScaler()

# Normalize the numeric columns
normalized_traindf = filled_traindf.copy()
normalized_traindf[numeric_columns.columns] = scaler.fit_transform(numeric_columns)


In [ ]:
normalized_traindf.info()

In [ ]:
plt.figure(figsize=(4,4))
ax = sns.barplot(x='is_ftp_login',y='counts',data=normalized_traindf.groupby('is_ftp_login').size().reset_index(name='counts'))
ax.set_title('Distribution of attack labels')
ax.set_xlabel('is_ftp_login')
ax.set_ylabel('Count')
plt.xticks(rotation=45)
plt.yticks(rotation=45)
plt.show()

In [ ]:
for column in traindf.columns:
    try:
        if traindf[column].dtype == type(object):
            le = LabelEncoder()
            traindf[column] = traindf[column].astype(str)
            traindf[column] = traindf[column].str.replace(".", "", regex=True)
            traindf[column] = le.fit_transform(traindf[column])
    except Exception as e:
        print(e)
        print(column)

C = traindf.iloc[1:, 43]
T = traindf.iloc[1:, 1:43]
scaler = Normalizer().fit(T)
trainT = scaler.transform(T)
y_train = np.array(C)
X_train = np.reshape(trainT, (trainT.shape[0], trainT.shape[1], 1))

In [ ]:
for column in testdf.columns:
    try:
        if testdf[column].dtype == type(object):
            le = LabelEncoder()
            testdf[column] = testdf[column].astype(str)
            testdf[column] = testdf[column].str.replace(".", "", regex=True)
            testdf[column] = le.fit_transform(testdf[column])
    except Exception as e:
        print(e)
        print(column)
C = testdf.iloc[1:, 43]
T = testdf.iloc[1:, 1:43]
scaler = Normalizer().fit(T)
testT = scaler.transform(T)
y_test = np.array(C)
X_test = np.reshape(testT, (testT.shape[0], testT.shape[1], 1))

In [ ]:
class RegularizationLayer(tf.keras.layers.Layer):
    def __init__(self, regularization_factor=0.01):
        super(RegularizationLayer, self).__init__()
        self.regularization_factor = regularization_factor

    def call(self, inputs):
        self.add_loss(tf.reduce_sum(tf.square(inputs)) * self.regularization_factor)
        return inputs


In [ ]:
y_train.shape

In [ ]:
X_test.shape

In [ ]:
lstm_output_size = 70
with STRATEGY.scope():
    cnn = Sequential([
        tf.keras.Input(shape=(42,1)),
        tf.keras.layers.BatchNormalization(momentum=0.95,name='stem_bn'),
        RegularizationLayer(regularization_factor=0.01),
        tf.keras.layers.GaussianNoise(0.1),
        tf.keras.layers.Convolution1D(64,3,padding='same',activation='relu'),
        tf.keras.layers.Convolution1D(64,3,padding='same',activation='relu'),
        tf.keras.layers.Dense(1, activation='relu'),
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(1,activation='sigmoid')
    ])
    cnn.compile(loss="binary_crossentropy", optimizer="adam", metrics=['binary_accuracy'])
    cnn.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[checkpointer, csv_logger, callbacks.EarlyStopping(monitor='accuracy', patience=3)])
    cnn.save("./content/results/cnn3results/cnn_model.hdf5")

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Split the dataset into features (X) and labels (y)
X = traindf.iloc[:, :-1]  # Features (all columns except the last one)
y = traindf.iloc[:, -1]   # Labels (last column)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the Decision Tree classifier
clf = DecisionTreeClassifier()

# Train the model
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


In [ ]:
import pandas as pd
import numpy as np
# Function to add random noise to numerical features
def add_random_noise(x, std_dev):
    noise = np.random.normal(0, std_dev, x.shape)
    return x + noise

# Function to replicate and perturb existing samples
def replicate_with_perturbation(x, num_replicas, perturbation_factor):
    replicas = np.tile(x, (num_replicas, 1))
    perturbations = np.random.uniform(-perturbation_factor, perturbation_factor, replicas.shape)
    replicas = replicas + perturbations
    return replicas

# Set the parameters for data augmentation
std_dev = 0.1  # Standard deviation for random noise addition
num_replicas = 3  # Number of replicas to create for each sample
perturbation_factor = 0.05  # Perturbation factor for replica perturbation

# Apply data augmentation techniques to traindf
augmented_data = []

# Iterate through each row in the DataFrame
for _, row in (traindf.iterrows()):
    # Add random noise to numerical features
    augmented_row = add_random_noise(row[:-1], std_dev)
    
    # Replicate and perturb the row
    replicas = replicate_with_perturbation(row[:-1], num_replicas, perturbation_factor)
    augmented_data.extend(replicas)
    
    augmented_data.append(augmented_row)

# Convert the augmented data back to a DataFrame
augmented_df = pd.DataFrame(augmented_data, columns=traindf.columns[:-1])

# Concatenate the augmented DataFrame with the original traindf
augmented_traindf = pd.concat([traindf, augmented_df])

# Shuffle the augmented traindf if desired
augmented_traindf = augmented_traindf.sample(frac=1).reset_index(drop=True)


In [ ]:
augmented_traindf.fillna(0, inplace=True)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Split the dataset into features (X) and labels (y)
X = augmented_traindf.iloc[:, :-1]  # Features (all columns except the last one)
y = augmented_traindf.iloc[:, -1]  # Labels (last column)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Create the Decision Tree classifier
clf = DecisionTreeClassifier()

# Train the model
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
